In [25]:
# Import necessary modules from Keras library
from keras.preprocessing.image import load_img, img_to_array
from keras import backend as K

# Define file paths for the target and reference images
target_image_path = '/content/wallpaper.jpeg'
reference_image_path = '/content/images.jpeg'

# Load the size (width and height) of the target image
width, height = load_img(target_image_path).size

# Set the desired height for the output image
img_height = 800

# Calculate the corresponding width to maintain the aspect ratio
img_width = int(width * img_height / height)


In [26]:
import numpy as np

def preprocess_image(image_path):
    # Load image, resize to target dimensions, and convert to NumPy array
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)

    # Add an extra dimension to the array to match the expected input shape
    img = np.expand_dims(img, axis=0)

    # Preprocess the image for the VGG19 model
    img = vgg19.preprocess_input(img)

    return img

def deprocess_image(x):
    # Invert the preprocessing steps applied to the image in the VGG19 model
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68

    # Reorder the color channels and ensure pixel values are within [0, 255]
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')

    return x


In [27]:
from keras.applications import vgg19

# Convert target and reference images to Keras constants using the preprocessed data
target_image = K.constant(preprocess_image(target_image_path))
reference_image = K.constant(preprocess_image(reference_image_path))

# Placeholder for the combination image, which is the image being generated
combination_image = K.placeholder((1, img_height, img_width, 3))

# Concatenate the target, reference, and combination images along the batch axis
input_tensor = K.concatenate([target_image, reference_image, combination_image], axis=0)

# Create the VGG19 model using the specified input tensor and pre-trained weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)


In [28]:
def content_loss(base, combination):
    """
    Compute the content loss between the base and combination images.

    Args:
        base: Feature representation of the content image.
        combination: Feature representation of the combination image.

    Returns:
        Content loss.
    """
    return K.sum(K.square(combination - base))


def gram_matrix(x):
    """
    Compute the Gram matrix for a given tensor.

    Args:
        x: Input tensor.

    Returns:
        Gram matrix.
    """
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram


def style_loss(style, combination):
    """
    Compute the style loss between the style and combination images.

    Args:
        style: Feature representation of the style image.
        combination: Feature representation of the combination image.

    Returns:
        Style loss.
    """
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))


def total_variation_loss(x):
    """
    Compute the total variation loss for an image.

    Args:
        x: Input image.

    Returns:
        Total variation loss.
    """
    a = K.square(
            x[:, :img_height - 1, :img_width - 1, :] -
            x[:, 1:, :img_width - 1, :])
    b = K.square(
            x[:, :img_height - 1, :img_width - 1, :] -
            x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [29]:
# Create a dictionary mapping layer names to their corresponding outputs in the model
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# Specify the content layer for content loss computation
content_layer = 'block5_conv2'

# Specify the style layers for style loss computation
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

# Define weights for total variation, style, and content losses
total_variation_weight = 1e-5
style_weight = 1
content_weight = 0.0003125


In [30]:
# Initialize the total loss variable
loss = K.variable(0.)

# Extract the features of the target and combination images at the specified content layer
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]

# Calculate and add the content loss to the total loss
loss = loss + content_weight * content_loss(target_image_features, combination_features)

# Iterate over the specified style layers to calculate and add style losses to the total loss
for layer_name in style_layers:
    # Extract features of the reference and combination images at the current style layer
    layer_features = outputs_dict[layer_name]
    reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]

    # Calculate the style loss and add it to the total loss
    sl = style_loss(reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl

# Add the total variation loss to the total loss
loss += total_variation_weight * total_variation_loss(combination_image)


In [31]:
# Import the TensorFlow library
import tensorflow as tf

# Import the disable_eager_execution function from the TensorFlow framework
from tensorflow.python.framework.ops import disable_eager_execution

# Disable eager execution in TensorFlow
disable_eager_execution()

In [32]:
# Compute the gradient of the total loss with respect to the combination image
grads = K.gradients(loss, combination_image)[0]

# Create a Keras function to fetch both the total loss and gradients
fetch_loss_and_grads = K.function([combination_image], [loss, grads])

In [ ]:
class Evaluator(object):
    """
    Evaluator class to compute loss and gradients for the combination image.

    Attributes:
        loss_value: The computed loss value.
        grad_values: The computed gradient values.
    """

    def __init__(self):
        self.loss_value = None
        self.grad_values = None

    def loss(self, x):
        """
        Compute the loss and gradients for the given combination image.

        Args:
            x: Combination image.

        Returns:
            loss_value: Computed loss value.
        """
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])

        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        """
        Return the stored gradient values.

        Args:
            x: Combination image.

        Returns:
            grad_values: Stored gradient values.
        """
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values


# Create an instance of the Evaluator class
evaluator = Evaluator()

# Import necessary modules for optimization
from scipy.optimize import fmin_l_bfgs_b
import imageio
import time

# Define parameters for optimization
result_prefix = 'my_result'
iterations = 20

# Initialize the combination image as the preprocessed target image
x = preprocess_image(target_image_path)
x = x.flatten()

# Perform optimization using L-BFGS
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()

    # Use L-BFGS to minimize the loss and update the combination image
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime=evaluator.grads, maxfun=20)

    print('Current loss value:', min_val)

    # Reshape and deprocess the optimized image
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)

    # Save the image
    fname = result_prefix + '_at_iteration_%d.png' % i
    if i == iterations - 1:
        imageio.imsave(fname, img)
    print('Image saved as', fname)

    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))
